In [27]:
# MIS 581
# Capstone - Whether Quality of Institution or Field of Study Better Predicts Student Loan Repayment
# Graham Burns
# Field of Study Analysis

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE

# Load dataset
data_path = '/content/Most-Recent-Cohorts-Field-of-Study.csv'
data = pd.read_csv(data_path, low_memory=False)

# Select relevant columns
columns = ['BBRR3_FED_COMP_PAIDINFULL', 'CIPCODE']
data = data[columns].dropna()

# Rename them for clarity
data.rename(columns={'BBRR3_FED_COMP_PAIDINFULL': 'loan_repaid_fully',
                     'CIPCODE': 'area_of_study'}, inplace=True)

# Convert CIPCODE to broad areas of study (2-digit level)
data['area_of_study'] = data['area_of_study'].astype(str).str[:2]

# Convert categorical variables to numeric for Area of Study
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_areas = encoder.fit_transform(data[['area_of_study']])
area_of_study_columns = encoder.get_feature_names_out(['area_of_study'])
data_encoded_areas = pd.DataFrame(encoded_areas, columns=area_of_study_columns)

# Remove original categorical column and merge encoded columns
data = data.drop(columns=['area_of_study'])
data = pd.concat([data, data_encoded_areas], axis=1)

# Convert loan repayment column to numeric
data['loan_repaid_fully'] = pd.to_numeric(data['loan_repaid_fully'], errors='coerce')

# Create binary target variable
data['loan_repaid'] = (data['loan_repaid_fully'] > 0).astype(int)

# Drop the original repayment column
X = data.drop(columns=['loan_repaid_fully', 'loan_repaid'])
y = data['loan_repaid']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Address class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Train Logistic Regression Model
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)

# Train Random Forest Model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Train XGBoost Model
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

# Model evaluation
def evaluate_model(y_true, y_pred, model_name):
    print(f"{model_name} Accuracy: {accuracy_score(y_true, y_pred):.3f}")
    print(f"{model_name} Precision: {precision_score(y_true, y_pred, zero_division=1):.3f}")
    print(f"{model_name} Recall: {recall_score(y_true, y_pred, zero_division=1):.3f}")
    print(f"{model_name} ROC AUC: {roc_auc_score(y_true, y_pred):.3f}\n")

evaluate_model(y_test, y_pred_logreg, "Logistic Regression")
evaluate_model(y_test, y_pred_rf, "Random Forest")
evaluate_model(y_test, y_pred_xgb, "XGBoost")

# Feature importance
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': rf.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Display top features
print("Top Features Influencing Loan Repayment:")
print(feature_importances.head(10))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:03:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Logistic Regression Accuracy: 0.657
Logistic Regression Precision: 0.008
Logistic Regression Recall: 0.931
Logistic Regression ROC AUC: 0.794

Random Forest Accuracy: 0.657
Random Forest Precision: 0.008
Random Forest Recall: 0.931
Random Forest ROC AUC: 0.794

XGBoost Accuracy: 0.657
XGBoost Precision: 0.008
XGBoost Recall: 0.931
XGBoost ROC AUC: 0.794

Top Features Influencing Loan Repayment:
             Feature  Importance
38  area_of_study_51    0.198458
39  area_of_study_52    0.176551
2   area_of_study_12    0.168490
37  area_of_study_50    0.045784
3   area_of_study_13    0.039379
32  area_of_study_45    0.032426
30  area_of_study_43    0.032043
34  area_of_study_47    0.030807
31  area_of_study_44    0.029487
4   area_of_study_14    0.026913


In [13]:
# MIS 581
# Capstone - Whether Quality of Institution or Field of Study Better Predicts Student Loan Repayment
# Graham Burns
# Quality of Institution Analysis against other Factors

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

# Load dataset
data_path = '/content/Most-Recent-Cohorts-Field-of-Study.csv'
data = pd.read_csv(data_path, low_memory=False)

# Select relevant columns
columns = ['BBRR3_FED_COMP_PAIDINFULL', 'BBRR3_FED_COMP_MAKEPROG', 'CIPCODE',
           'EARN_MDN_HI_1YR', 'EARN_MDN_HI_2YR', 'DEBT_ALL_PP_ANY_MDN', 'DEBT_ALL_PP_ANY_MDN10YRPAY']
data = data[columns].dropna()

# Rename them for clarity
data.rename(columns={'BBRR3_FED_COMP_PAIDINFULL': 'loan_repaid_fully',
                     'BBRR3_FED_COMP_MAKEPROG': 'loan_repaid_progress',
                     'CIPCODE': 'area_of_study',
                     'EARN_MDN_HI_1YR': 'median_earnings_1yr',
                     'EARN_MDN_HI_2YR': 'median_earnings_2yr',
                     'DEBT_ALL_PP_ANY_MDN': 'graduate_median_debt',
                     'DEBT_ALL_PP_ANY_MDN10YRPAY': 'monthly_loan_payment'}, inplace=True)

# Convert CIPCODE to broad areas of study (2-digit level)
data['area_of_study'] = data['area_of_study'].astype(str).str[:2]

# Convert categorical variables to numeric for Area of Study
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_areas = encoder.fit_transform(data[['area_of_study']])
area_of_study_columns = encoder.get_feature_names_out(['area_of_study'])
data_encoded_areas = pd.DataFrame(encoded_areas, columns=area_of_study_columns)

# Remove original categorical column and merge encoded columns
data = data.drop(columns=['area_of_study'])
data = pd.concat([data, data_encoded_areas], axis=1)

# Convert loan repayment columns to numeric
data['loan_repaid_fully'] = pd.to_numeric(data['loan_repaid_fully'], errors='coerce')
data['loan_repaid_progress'] = pd.to_numeric(data['loan_repaid_progress'], errors='coerce')

# Create binary target variable
data['loan_repaid'] = ((data['loan_repaid_fully'] > 0) | (data['loan_repaid_progress'] > 0)).astype(int)

# Define numeric features
numeric_features = ['median_earnings_1yr', 'median_earnings_2yr', 'graduate_median_debt', 'monthly_loan_payment']
for feature in numeric_features:
    data[feature] = data[feature].replace(['PrivacySuppressed', 'PS'], np.nan)
    data[feature] = pd.to_numeric(data[feature], errors='coerce')

# Drop rows with NaN values
data = data.dropna(subset=numeric_features)

# Scale numeric features
scaler = MinMaxScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

# Split dataset
X = data.drop(columns=['loan_repaid_fully', 'loan_repaid_progress', 'loan_repaid'])
y = data['loan_repaid']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression Model
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)

# Train Random Forest Model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Model evaluation
def evaluate_model(y_true, y_pred, model_name):
    print(f"{model_name} Accuracy: {accuracy_score(y_true, y_pred):.3f}")
    print(f"{model_name} Precision: {precision_score(y_true, y_pred):.3f}")
    print(f"{model_name} Recall: {recall_score(y_true, y_pred):.3f}")
    print(f"{model_name} ROC AUC: {roc_auc_score(y_true, y_pred):.3f}\n")

evaluate_model(y_test, y_pred_logreg, "Logistic Regression")
evaluate_model(y_test, y_pred_rf, "Random Forest")

# Feature importance
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': rf.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Display top features
print("Top Features Influencing Loan Repayment:")
print(feature_importances.head(10))

<ipython-input-13-ba90d9383d1c>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numeric_features] = scaler.fit_transform(data[numeric_features])


Logistic Regression Accuracy: 0.946
Logistic Regression Precision: 0.000
Logistic Regression Recall: 0.000
Logistic Regression ROC AUC: 0.500

Random Forest Accuracy: 0.988
Random Forest Precision: 0.962
Random Forest Recall: 0.810
Random Forest ROC AUC: 0.904

Top Features Influencing Loan Repayment:
                 Feature  Importance
2   graduate_median_debt    0.264057
1    median_earnings_2yr    0.231788
0    median_earnings_1yr    0.219295
3   monthly_loan_payment    0.204974
43      area_of_study_52    0.004920
5       area_of_study_11    0.004892
41      area_of_study_50    0.004702
42      area_of_study_51    0.004196
36      area_of_study_45    0.003907
31      area_of_study_40    0.003830


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
